In [1]:
from PyPDF2 import PdfFileReader, PdfFileWriter
from PIL import Image
from pdf2image import convert_from_path

import os
import pathlib
import pytesseract
import re
import time


In [9]:
input_dir  = "C:\\test\\input\\"
output_dir = "C:\\test\\output\\"

# Uncomment the line below to provide path to tesseract manually
# pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

In [10]:
#convert seconds to hr:min:sec format

def convert_to_hms(s):
    
    minutes, seconds = divmod(s, 60)
    hours, minutes   = divmod(minutes, 60)

    return print("%d:%02d:%02d" % (hours, minutes, seconds))

# secs = 1784
# convert_to_hms(secs)


In [11]:

# get the start time
start_time = int(time.perf_counter())

for root, folders, pdfs in os.walk(input_dir):

    print("Home Folder  : " + os.path.dirname(root))

    for pdf in pdfs:
    
        if not pdf.endswith('.pdf'): continue
        
        pdf_in     = open(root + "\\" + pdf, 'rb')
        pdf_reader = PdfFileReader(pdf_in, strict=True)

        output_fld = root.replace(input_dir, output_dir)
        input_pdf  = root + "\\" + pdf

        # ---------------------------------------------------------
        # if folder doesnt exist - create it
        # ---------------------------------------------------------
        if not os.path.exists(output_fld):
            os.makedirs(output_fld)

        pdf_out    = open(output_fld + "\\" + pdf, 'wb')
        pdf_writer = PdfFileWriter()

        print("Input PDF  : " + input_pdf)
        print("Output PDF : " + output_fld + "\\" + pdf)
    
        # ---------------------------------------------------------
        # loop through the PDF page by page and check page rotation
        # ---------------------------------------------------------
        for pagenum in range(pdf_reader.numPages):

            page = pdf_reader.getPage(pagenum)

            page_image = convert_from_path(input_pdf, dpi=300, first_page=pagenum+1, last_page=pagenum+1, poppler_path=r"C:\poppler-21.03.0\Library\bin")

            # save image out
            page_image[0].save('pdf_img_' + str(pagenum) + '.jpg', 'JPEG')
            
            # open image in paint
            # page_image[0].show()

            # rotation_data = pytesseract.image_to_osd('pdf_img_' + str(pagenum) + '.jpg', config="--tessdata-dir 'C:\\Program Files\\Tesseract-OCR\\tessdata' --psm 0 --dpi 300")

            try:
                rotation_data = pytesseract.image_to_osd('pdf_img_' + str(pagenum) + '.jpg', config="--tessdata-dir 'C:\\Program Files\\Tesseract-OCR\\tessdata' --psm 0 --dpi 300")
            except:
                print("Error Occured")
                print("Using 70 instead. Too few characters.")
                print("Skipping this page Error during processing.")
                continue

            print("==== ROTATION DATA ====================================")

            print(rotation_data)
        
            # extract the text rotation of the text on the page
            orientation   = re.search('(?<=Orientation in degrees: )\d+', rotation_data).group(0)
            rotation      = re.search('(?<=Rotate: )\d+', rotation_data).group(0)

            text_orientation = float(orientation)
            text_rotation    = float(rotation)

            print("Page : " + str(pagenum) + " Orientation in Degrees : " + str(orientation) + " Text Rotation : " + str(rotation))

            print("=======================================================")

            # get the rotation of the page
            # orientation = page.get('/Rotate')

            print("Page No : %d Text Rotation %s" %(pagenum, text_rotation))
            
            if text_orientation == 90:
                page.rotateClockwise(text_rotation)
            elif text_orientation == 180:
                page.rotateClockwise(text_rotation)
            elif text_orientation == 270:
                page.rotateClockwise(text_rotation)

            pdf_writer.addPage(page)

        # --------------------------------------------------------------------
        # When finished rotating all pages in PDF, write it to output location
        # ---------------------------------------------------------------------
        pdf_writer.write(pdf_out)

        # --------------------------------------------------------------------
        # Close the file handles
        # ---------------------------------------------------------------------
        pdf_out.close()
        pdf_in.close()

# get the end time
end_time = int(time.perf_counter())


uitment-1.pdf
==== ROTATION DATA ====================================
Page number: 0
Orientation in degrees: 180
Rotate: 180
Orientation confidence: 14.78
Script: Latin
Script confidence: 1.63

Page : 0 Orientation in Degrees : 180 Text Rotation : 180
Page No : 0 Text Rotation 180.0
Input PDF  : C:\test\input\Ossian McCullock\Recruitment\Recruitment-2.pdf
Output PDF : C:\test\output\Ossian McCullock\Recruitment\Recruitment-2.pdf
==== ROTATION DATA ====================================
Page number: 0
Orientation in degrees: 180
Rotate: 180
Orientation confidence: 8.45
Script: Latin
Script confidence: 4.21

Page : 0 Orientation in Degrees : 180 Text Rotation : 180
Page No : 0 Text Rotation 180.0
Input PDF  : C:\test\input\Ossian McCullock\Recruitment\Recruitment-3.pdf
Output PDF : C:\test\output\Ossian McCullock\Recruitment\Recruitment-3.pdf
==== ROTATION DATA ====================================
Page number: 0
Orientation in degrees: 270
Rotate: 90
Orientation confidence: 25.04
Script: L

In [12]:
pdf_out.close()
pdf_in.close()

print("==== Execution Time ====================================")
convert_to_hms(end_time - start_time)
print("=======================================================")


==== Execution Time ====================================
0:31:44



131 Files, 38 Folders in 32 minutes

os.walk returns tuple (working folder, list of subfolders, list of files in working folder)  
str.replace("text", "text")  
https://i.ibb.co/27bKNJ8/book-rot2.jpg  

poppler_path = r"C:\path\to\poppler-xx\bin" as an argument in convert_from_path.  

pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"

images = pdf2image.convert_from_path("test2.pdf", first_page=3, last_page=3,poppler_path=r"C:\poppler-21.03.0\Library\bin")

images[0].show()

images = convert_from_path('example.pdf') #Read pdf file
for i in range(len(images)):
          images[i].save('img'+str(i)+'.jpg', 'JPEG')
